In [1]:
import pandas as pd

In [2]:
Spa1_league = pd.read_csv("Spa1_league.csv")

In [3]:
Spa1_league = Spa1_league.drop('Unnamed: 0', axis=1)
Spa1_league = Spa1_league.drop('index', axis = 1)
Spa1_league.head()

,League,Season,Home_Team,Away_Team,Home_Goals,Away_Goals,Home_FH,Away_FH,Home_Win,Draw,Away_Win,Over_2_5,Under_2_5,Code_1,Code_2,Total_Goals,FH_Goals,Winner,HT_Winner
0,Spa1,20,Bilbao,Barcelona,1,0,0,0,5340.0,3620.0,1780.0,1980,1930,3008,3018,1,0,H,D
1,Spa1,20,Mallorca,Eibar,2,1,1,0,2820.0,3190.0,2770.0,2430,1600,3009,3039,3,1,H,H
2,Spa1,20,Valencia,Real Sociedad,1,1,0,0,1690.0,3880.0,5470.0,2060,1850,3019,3015,2,0,D,D
3,Spa1,20,Villarreal,Granada,4,4,1,1,1540.0,4190.0,6870.0,1950,1950,3020,3033,8,2,D,D
4,Spa1,20,Celta Vigo,Real Madrid,1,3,0,1,5100.0,4460.0,1650.0,1490,2760,3022,3010,4,1,A,A


In [4]:
matchday = []
for j in range(0,4):
    s = 20 + j
    for i in range(len(Spa1_league.query(f"Season == {s}"))):
        md = (i//10) + 1
        matchday.append(md)
if "Matchday" not in Spa1_league.columns:
    Spa1_league.insert(2, "Matchday", matchday, True)

In [5]:
for s in Spa1_league['Season'].unique():
    globals()["Madrid_" + str(s)] = Spa1_league.query(f"Season == {s}").query("Home_Team == 'Real Madrid' or Away_Team == 'Real Madrid'").reset_index()

In [6]:
def Data(Madrid_2022):
    
    Madrid_2022 = Madrid_2022.drop('index', axis = 1)
    
    for i in Madrid_2022.index:
        Madrid_2022['Matchday'].iloc[i] = i+1
    
    #==================================================
    # If Madrid Won, Draw or Lose the game
    #==================================================
    
    mwinner = [] 
    
    for i in range(len(Madrid_2022)):
        if Madrid_2022['Winner'].iloc[i] == 'H':
            if Madrid_2022['Code_1'].iloc[i] == 3010:
                mwinner.append('W')
            else:
                mwinner.append('L')
        elif Madrid_2022['Winner'].iloc[i] == 'A':
            if Madrid_2022['Code_2'].iloc[i] == 3010:
                mwinner.append('W')
            else:
                mwinner.append('L')
        else:
            mwinner.append('D')
    
    #==================================================
    # If madrid was Winning, Drawing or Lossing at First Half
    #==================================================
    mwinner_ht = [] 
    for i in range(len(Madrid_2022)):
        if Madrid_2022['HT_Winner'].iloc[i] == 'H':
            if Madrid_2022['Code_1'].iloc[i] == 3010:
                mwinner_ht.append('W')
            else:
                mwinner_ht.append('L')
        elif Madrid_2022['HT_Winner'].iloc[i] == 'A':
            if Madrid_2022['Code_2'].iloc[i] == 3010:
                mwinner_ht.append('W')
            else:
                mwinner_ht.append('L')
        else:
            mwinner_ht.append('D')
    
    #==================================================
    # If Madrid was at home or not
    #==================================================
    
    h_or_a = []
    for i in range(len(Madrid_2022)):
        if Madrid_2022['Code_1'].iloc[i] == 3010:
            h_or_a.append(1)
        else:
            h_or_a.append(0)
    
    #==================================================
    # Madrid Rival
    #==================================================
    
    m_rival = []
    for i in range(len(h_or_a)):
        if h_or_a[i] == 1:
            m_rival.append(Madrid_2022['Away_Team'].iloc[i])
        else:
            m_rival.append(Madrid_2022['Home_Team'].iloc[i])
    
    #==================================================
    # Madrid and Rival Goals
    #==================================================
    
    m_goals = []
    for i in range(len(h_or_a)):
        if h_or_a[i] == 1:
            m_goals.append(Madrid_2022['Home_Goals'].iloc[i])
        else:
            m_goals.append(Madrid_2022['Away_Goals'].iloc[i])
            
    rival_goals = []
    for i in range(len(h_or_a)):
        if h_or_a[i] == 1:
            rival_goals.append(Madrid_2022['Away_Goals'].iloc[i])
        else:
            rival_goals.append(Madrid_2022['Home_Goals'].iloc[i])
            
    #==================================================
    # Madrid and Rival Goals at First Half
    #==================================================
    
    fh_m_goals = []
    for i in range(len(h_or_a)):
        if h_or_a[i] == 1:
            fh_m_goals.append(Madrid_2022['Home_FH'].iloc[i])
        else:
            fh_m_goals.append(Madrid_2022['Away_FH'].iloc[i])
            
    fh_r_goals = []
    for i in range(len(h_or_a)):
        if h_or_a[i] == 1:
            fh_r_goals.append(Madrid_2022['Away_FH'].iloc[i])
        else:
            fh_r_goals.append(Madrid_2022['Home_FH'].iloc[i])
            
    #==================================================
    # Odds that Madrid Wons, Draw, or Lose
    #==================================================
    
    odds_m_win = []
    for i in range(len(h_or_a)):
        if h_or_a[i] == 1:
            odds_m_win.append(Madrid_2022['Home_Win'].iloc[i])
        else:
            odds_m_win.append(Madrid_2022['Away_Win'].iloc[i])
            
    odds_m_lose = []
    for i in range(len(h_or_a)):
        if h_or_a[i] == 1:
            odds_m_lose.append(Madrid_2022['Away_Win'].iloc[i])
        else:
            odds_m_lose.append(Madrid_2022['Home_Win'].iloc[i])
    
    #==================================================
    # ADDING NEW FEATURES AND REMOVING OLD
    #==================================================
    
    #Results
    if "Result" not in Madrid_2022.columns:
        Madrid_2022.insert(19, "Result", mwinner, True)
    Madrid_2022 = Madrid_2022.drop('Winner', axis = 1)
    
    #Results at Half Time
    if "Result_HT" not in Madrid_2022.columns:
        Madrid_2022.insert(18, "Result_HT", mwinner_ht, True)
    Madrid_2022 = Madrid_2022.drop('HT_Winner', axis = 1)
    
    #Madrid was at home or not
    if "Home" not in Madrid_2022.columns:
        Madrid_2022.insert(3, "Home", h_or_a, True)
    Madrid_2022 = Madrid_2022.drop('Home_Team', axis = 1)
    Madrid_2022 = Madrid_2022.drop('Away_Team', axis = 1)
    
    #Madrid rival
    if "Rival" not in Madrid_2022.columns:
        Madrid_2022.insert(4, "Rival", m_rival, True)
        
    #Madrid Goals
    if "Madrid_Goals" not in Madrid_2022.columns:
        Madrid_2022.insert(5, "Madrid_Goals", m_goals, True)
        
    #Rival Goals
    if "Rival_Goals" not in Madrid_2022.columns:
        Madrid_2022.insert(6, "Rival_Goals", rival_goals, True)
        
    Madrid_2022 = Madrid_2022.drop('Home_Goals', axis = 1)
    Madrid_2022 = Madrid_2022.drop('Away_Goals', axis = 1)
    Madrid_2022 = Madrid_2022.drop('Home_FH', axis = 1)
    Madrid_2022 = Madrid_2022.drop('Away_FH', axis = 1)
    Madrid_2022 = Madrid_2022.drop('Home_Win', axis = 1)
    Madrid_2022 = Madrid_2022.drop('Away_Win', axis = 1)
    Madrid_2022 = Madrid_2022.drop('Code_1', axis = 1)
    Madrid_2022 = Madrid_2022.drop('Code_2', axis = 1)
    
    #Madrid goals at First Half
    if "Madrid_Goals_FH" not in Madrid_2022.columns:
        Madrid_2022.insert(7, "Madrid_Goals_FH", fh_m_goals, True)

    #Rival Goals at First Half
    fh_r_goals
    if "Rival_Goals_FH" not in Madrid_2022.columns:
        Madrid_2022.insert(8, "Rival_Goals_FH", fh_r_goals, True)
    
    #Odds of Madrid Wins or Lose
    if "Madrid_Win" not in Madrid_2022.columns:
        Madrid_2022.insert(9, "Madrid_Win", odds_m_win, True)
    if "Madrid_Lose" not in Madrid_2022.columns:
        Madrid_2022.insert(11, "Madrid_Lose", odds_m_lose, True)
    
    points = []
    p = 0
    for i in range(len(Madrid_2022)):
        if Madrid_2022['Result'].iloc[i] == 'W':
            p += 3
            points.append(p)
        elif Madrid_2022['Result'].iloc[i] == 'D':
            p += 1
            points.append(p)
        else:
            p += 0
            points.append(p)
            
    if "Points" not in Madrid_2022.columns:
        Madrid_2022.insert(18, "Points", points, True)
        
    return Madrid_2022

In [7]:
Madrid_2020 = Data(Madrid_20)

/var/folders/c5/d9gjzjp97kxc9lwztd1vtkn00000gn/T/ipykernel_67508/4080284478.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Madrid_2022['Matchday'].iloc[i] = i+1
/var/folders/c5/d9gjzjp97kxc9lwztd1vtkn00000gn/T/ipykernel_67508/4080284478.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Madrid_2022['Matchday'].iloc[i] = i+1
/var/folders/c5/d9gjzjp97kxc9lwztd1vtkn00000gn/T/ipykernel_67508/4080284478.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

In [8]:
Madrid_2021 = Data(Madrid_21)

/var/folders/c5/d9gjzjp97kxc9lwztd1vtkn00000gn/T/ipykernel_67508/4080284478.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Madrid_2022['Matchday'].iloc[i] = i+1
/var/folders/c5/d9gjzjp97kxc9lwztd1vtkn00000gn/T/ipykernel_67508/4080284478.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Madrid_2022['Matchday'].iloc[i] = i+1
/var/folders/c5/d9gjzjp97kxc9lwztd1vtkn00000gn/T/ipykernel_67508/4080284478.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

In [9]:
Madrid_2022 = Data(Madrid_22)

/var/folders/c5/d9gjzjp97kxc9lwztd1vtkn00000gn/T/ipykernel_67508/4080284478.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Madrid_2022['Matchday'].iloc[i] = i+1
/var/folders/c5/d9gjzjp97kxc9lwztd1vtkn00000gn/T/ipykernel_67508/4080284478.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Madrid_2022['Matchday'].iloc[i] = i+1
/var/folders/c5/d9gjzjp97kxc9lwztd1vtkn00000gn/T/ipykernel_67508/4080284478.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

In [10]:
Madrid_2023 = Data(Madrid_23)

/var/folders/c5/d9gjzjp97kxc9lwztd1vtkn00000gn/T/ipykernel_67508/4080284478.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Madrid_2022['Matchday'].iloc[i] = i+1
/var/folders/c5/d9gjzjp97kxc9lwztd1vtkn00000gn/T/ipykernel_67508/4080284478.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Madrid_2022['Matchday'].iloc[i] = i+1
/var/folders/c5/d9gjzjp97kxc9lwztd1vtkn00000gn/T/ipykernel_67508/4080284478.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

In [11]:
Madrid_2020.tail()

,League,Season,Matchday,Home,Rival,Madrid_Goals,Rival_Goals,Madrid_Goals_FH,Rival_Goals_FH,Madrid_Win,Draw,Madrid_Lose,Over_2_5,Under_2_5,Total_Goals,FH_Goals,Result_HT,Result,Points
33,Spa1,20,34,0,Bilbao,1,0,0,0,1830.0,3560.0,4900.0,2090,1810,1,0,D,W,77
34,Spa1,20,35,1,Alaves,2,0,1,0,1210.0,6600.0,15640.0,1660,2300,2,1,W,W,80
35,Spa1,20,36,0,Granada,2,1,2,0,1430.0,4600.0,8270.0,1790,2110,3,2,W,W,83
36,Spa1,20,37,1,Villarreal,2,1,1,0,1420.0,5120.0,7330.0,1560,2540,3,1,W,W,86
37,Spa1,20,38,0,Leganes,2,2,1,1,2140.0,3760.0,3340.0,1980,1920,4,2,D,D,87


In [12]:
Madrid_2021.tail()

,League,Season,Matchday,Home,Rival,Madrid_Goals,Rival_Goals,Madrid_Goals_FH,Rival_Goals_FH,Madrid_Win,Draw,Madrid_Lose,Over_2_5,Under_2_5,Total_Goals,FH_Goals,Result_HT,Result,Points
33,Spa1,21,34,1,Osasuna,2,0,0,0,1360.0,4750.0,8500.0,1730,2100,2,0,D,W,74
34,Spa1,21,35,1,FC Sevilla,2,2,0,1,1950.0,3790.0,4030.0,1990,1920,4,1,L,D,75
35,Spa1,21,36,0,Granada,4,1,2,0,1410.0,5230.0,7910.0,1690,2280,5,2,W,W,78
36,Spa1,21,37,0,Bilbao,1,0,0,0,1560.0,4460.0,6110.0,1770,2160,1,0,D,W,81
37,Spa1,21,38,1,Villarreal,2,1,0,1,1600.0,4650.0,5430.0,1550,2610,3,1,L,W,84


In [13]:
Madrid_2022.tail()

,League,Season,Matchday,Home,Rival,Madrid_Goals,Rival_Goals,Madrid_Goals_FH,Rival_Goals_FH,Madrid_Win,Draw,Madrid_Lose,Over_2_5,Under_2_5,Total_Goals,FH_Goals,Result_HT,Result,Points
33,Spa1,22,34,1,Espanyol,4,0,2,0,1480.0,4850.0,7040.0,1560,2580,4,2,W,W,81
34,Spa1,22,35,0,Atletico,0,1,0,1,4250.0,3520.0,1980.0,1990,1920,1,1,L,L,81
35,Spa1,22,36,1,Levante,6,0,4,0,1500.0,5200.0,5980.0,1390,3160,6,4,W,W,84
36,Spa1,22,37,0,Cadiz,1,1,1,1,2210.0,3220.0,3470.0,2020,1810,2,2,D,D,85
37,Spa1,22,38,1,Betis,0,0,0,0,1680.0,4490.0,4690.0,1390,3130,0,0,D,D,86


In [14]:
Madrid_2023.tail()

,League,Season,Matchday,Home,Rival,Madrid_Goals,Rival_Goals,Madrid_Goals_FH,Rival_Goals_FH,Madrid_Win,Draw,Madrid_Lose,Over_2_5,Under_2_5,Total_Goals,FH_Goals,Result_HT,Result,Points
33,Spa1,23,34,1,Getafe,1,0,0,0,1600.0,4110.0,6320.0,2010,1900,1,0,D,W,71
34,Spa1,23,35,0,Valencia,0,1,0,1,2250.0,3720.0,3240.0,1730,2220,1,1,L,L,71
35,Spa1,23,36,1,Vallecano,2,1,1,0,1410.0,5140.0,8140.0,1540,2610,3,1,W,W,74
36,Spa1,23,37,0,FC Sevilla,2,1,1,1,2310.0,3580.0,3220.0,1790,2120,3,2,D,W,77
37,Spa1,23,38,1,Bilbao,1,1,0,0,1680.0,4380.0,4840.0,1560,2530,2,0,D,D,78


In [15]:
Madrid_merged = pd.concat([Madrid_2020, Madrid_2021, Madrid_2022, Madrid_2023], axis = 0).reset_index()

In [16]:
Madrid_merged = Madrid_merged.drop('index', axis = 1)
Madrid_merged = Madrid_merged.drop('League', axis = 1)
Madrid_merged

,Season,Matchday,Home,Rival,Madrid_Goals,Rival_Goals,Madrid_Goals_FH,Rival_Goals_FH,Madrid_Win,Draw,Madrid_Lose,Over_2_5,Under_2_5,Total_Goals,FH_Goals,Result_HT,Result,Points
0,20,1,0,Celta Vigo,3,1,1,0,1650.0,4460.0,5100.0,1490,2760,4,1,W,W,3
1,20,2,1,Valladolid,1,1,0,0,1230.0,7320.0,13120.0,1400,3100,2,0,D,D,4
2,20,3,0,Villarreal,2,2,1,1,1940.0,4070.0,3730.0,1400,3110,4,2,D,D,5
3,20,4,1,Levante,3,2,3,0,1270.0,6800.0,9990.0,1260,4150,5,3,W,W,8
4,20,5,0,FC Sevilla,1,0,0,0,2640.0,3780.0,2580.0,1500,2720,1,0,D,W,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,23,34,1,Getafe,1,0,0,0,1600.0,4110.0,6320.0,2010,1900,1,0,D,W,71
148,23,35,0,Valencia,0,1,0,1,2250.0,3720.0,3240.0,1730,2220,1,1,L,L,71
149,23,36,1,Vallecano,2,1,1,0,1410.0,5140.0,8140.0,1540,2610,3,1,W,W,74
150,23,37,0,FC Sevilla,2,1,1,1,2310.0,3580.0,3220.0,1790,2120,3,2,D,W,77


In [17]:
Madrid_merged.to_pickle("01_data_interim.pkl")